In [ ]:
from vllm import LLM, SamplingParams

In [ ]:
prompts = [
    "Translate the text English to Spanish: Hello, my name is Jemmy.",
    "Translate the text English to Spanish: The president of the United States is Kane.",
    "Translate the text English to Spanish: The capital of France is Paris",
    "Translate the text English to Spanish: The future of AI is bright.",
]
sampling_params = SamplingParams(temperature=1.0, top_p=0.95)

In [ ]:
llm = LLM(model="unsloth/Qwen2.5-32B-Instruct",dtype="half",tensor_parallel_size=4)

In [ ]:
outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
COD_plus={
    "prompt1.1":"Expand this chain of dictionary translations by adding missing languages where appropriate: ",
    "prompt1.2": "Identify any missing translations in this language chain and complete them: ",
    "prompt2.1":"For each word in this chain of dictionary translations, add appropriate synonyms in parentheses: ",
    "prompt2.2": "Enhance this dictionary chain by inserting synonyms for each term in the respective language: ",
    "prompt3.1":"Expand this dictionary chain by adding one-hop related concepts from a knowledge graph. Include medical, biological, or psychological terms where relevant: ",
    "prompt3.2": "Enhance this chain of dictionary translations by inserting related concepts (such as medical conditions, biological processes, or symptoms) at each step: "
}
for x in COD_plus:
    print(x)

In [ ]:
import json

file_path = '/home/mshahidul/project1/results_new/prompt_extention_testing.json'

with open(file_path, 'r') as file:
    data = json.load(file)

print(data[2]['text'])

In [ ]:
file_path_medline = '/home/mshahidul/project1/results_new/Medline/Qwen2.5-14B-Instruct_without_finetuned_medline.json'
import json
with open(file_path_medline, 'r') as file:
    medline_data = json.load(file)
    columns = medline_data[0].keys()
    print(columns)
sum([x['bleu_score']['bleu_score'] for x in (medline_data)])/len(medline_data)


In [ ]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
# from google import genai
import json
client = OpenAI(api_key=json.load(open('/home/mshahidul/project1/api.json', 'r'))['openai_api'])
# client = genai.Client(api_key=json.load(open('/home/mshahidul/project1/api.json', 'r'))['gemini_api'])
url = "http://sirchus.com/gpu_status.txt"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    text = response.text
    txt=f'''
        {text}

        Find the free available GPU (GPU Memory usage<=10%) using above context. Sort available GPU based on high configuration to low configuration. Show this table format.
        Add gpu information:
        GPU server 1 (omega):IP: 172.16.34.1
        GPU server 2 (alpha):IP: 172.16.34.21
        GPU server 3 (beta): IP: 172.16.34.22 
        GPU server 4 (gamma): IP: 172.16.34.29
        '''
    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                    {"role": "user", "content": f"{txt}"}],
            temperature=0.5
        )
    ans=(response.choices[0].message.content)
    
    print(ans)

In [ ]:
from google import genai
import requests
from bs4 import BeautifulSoup
import json
client = genai.Client(api_key=json.load(open('/home/mshahidul/project1/api.json', 'r'))['gemini_api'])
url = "http://sirchus.com/gpu_status.txt"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    text = response.text
    txt=f'''
        {text}

        Find the free available GPU (GPU Memory usage<=10%) using above context. Sort available GPU based on high configuration to low configuration. Show this table format.
        Add gpu information:
        GPU server 1 (omega):IP: 172.16.34.1
        GPU server 2 (alpha):IP: 172.16.34.21
        GPU server 3 (beta): IP: 172.16.34.22 
        GPU server 4 (gamma): IP: 172.16.34.29
        '''
    
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"{txt}",
    )
    print(response.text)

In [ ]:
import os
import json
from groq import Groq
import requests
client = Groq(
    api_key=json.load(open('/home/mshahidul/project1/api.json', 'r'))['groq_api'],
)
url = "http://sirchus.com/gpu_status.txt"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    text = response.text
    txt=f'''
        {text}

        Find the free available GPU (GPU Memory usage<=10%) using above context. Sort available GPU based on high configuration to low configuration. Show this table format.
        Add gpu information:
        GPU server 1 (omega):IP: 172.16.34.1
        GPU server 2 (alpha):IP: 172.16.34.21
        GPU server 3 (beta): IP: 172.16.34.22 
        GPU server 4 (gamma): IP: 172.16.34.29
        '''
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"{txt}",
            }
        ],
        model="llama-3.3-70b-versatile",
    )

    print(chat_completion.choices[0].message.content)

In [ ]:
import json
file_path = "/home/mshahidul/project1/all_tran_data/dataset/Sampled_100_MedlinePlus_eng_spanish_pair.json"
with open(file_path, 'r', encoding='utf-8') as json_file:
    original_file = json.load(json_file)
with open('/home/mshahidul/project1/all_tran_data/updated_keywords.json', 'r', encoding='utf-8') as keywords_file:
    keywords = json.load(keywords_file)
ans=0
not_exist_count = 0
xx=[x['sentence'] for x in (keywords)]
for x in original_file:
    if x['english'] not in xx:
        print(x['english'])
        not_exist_count+=1
# print(not_exist_count)

In [ ]:
from sentence_transformers import SentenceTransformer, util
import json

# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load original dataset
file_path = "/home/mshahidul/project1/all_tran_data/dataset/Sampled_100_MedlinePlus_eng_spanish_pair.json"
with open(file_path, 'r', encoding='utf-8') as json_file:
    original_file = json.load(json_file)

# Load keywords dataset
keywords_path = "/home/mshahidul/project1/all_tran_data/keywords.json"
with open(keywords_path, 'r', encoding='utf-8') as keywords_file:
    keywords = json.load(keywords_file)

# Extract sentences
original_sentences = [x['english'] for x in original_file]
keyword_sentences = [x['sentence'] for x in keywords]

# Compute embeddings
original_embeddings = model.encode(original_sentences, convert_to_tensor=True)
keyword_embeddings = model.encode(keyword_sentences, convert_to_tensor=True)

# Perform similarity check and replace if similarity > 90%
threshold = 0.90
for i, kw_emb in enumerate(keyword_embeddings):
    similarities = util.cos_sim(kw_emb, original_embeddings)
    max_sim_idx = similarities.argmax().item()
    max_sim_value = similarities[0, max_sim_idx].item()

    if max_sim_value > threshold:
        keywords[i]['sentence'] = original_sentences[max_sim_idx]  # Replace sentence

# Save updated keywords file
updated_keywords_path = "/home/mshahidul/project1/all_tran_data/updated_keywords.json"
with open(updated_keywords_path, 'w', encoding='utf-8') as updated_file:
    json.dump(keywords, updated_file, ensure_ascii=False, indent=4)

print(f"Updated keyword sentences saved to {updated_keywords_path}")


In [ ]:
import mysql.connector
import json
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from openai import OpenAI  # Assuming OpenAI API is set up

# Database connection details
db_config = {
    'host': '172.16.34.1',
    'port': 3307,
    'user': 'umls',
    'password': 'umls',
    'database': 'umls2024'
}


In [ ]:
keyword="congenital heart surgery"
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor(dictionary=True)
cursor.execute("SELECT CUI FROM MRCONSO WHERE STR LIKE %s LIMIT 1", (f"%{keyword}%",))
result = cursor.fetchone()
if result is None:
    print(f"No CUI found for keyword: {keyword}")

cui = result["CUI"]

print(f"Keyword: {keyword}, CUI: {cui}")
cursor.execute("SELECT LAT, STR FROM MRCONSO WHERE CUI = %s AND LAT IN (%s, %s, %s)",
                       (cui, 'FRE', 'POR', 'GER'))
rows = cursor.fetchall()
rows

In [ ]:
keyword="heart surgery"
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor(dictionary=True)
cursor.execute("SELECT CUI FROM MRCONSO WHERE STR LIKE %s", (f"%{keyword}%",))
result = cursor.fetchone()
cui = result["CUI"]
print(f"Keyword: {keyword}, CUI: {cui}")
cursor.execute("SELECT LAT, STR FROM MRCONSO WHERE CUI = %s AND LAT IN (%s, %s, %s)",
                       (cui, 'FRE', 'POR', 'GER'))
rows = cursor.fetchall()
rows

In [ ]:
cursor.execute("SELECT DEF FROM MRDEF WHERE CUI = %s", (cui,))
rows = cursor.fetchall()
rows

In [ ]:
def get_cui_for_keyword(cursor, keyword):
    cursor.execute("SELECT CUI FROM MRCONSO WHERE STR LIKE %s LIMIT 1", (f"%{keyword}%",))
    return cursor.fetchone()

def get_translations(cursor, cui):
    cursor.execute("SELECT LAT, STR FROM MRCONSO WHERE CUI = %s AND LAT IN (%s, %s, %s)",
                   (cui, 'FRE', 'POR', 'GER'))
    return cursor.fetchall()

def find_and_merge_translations(cursor, keyword):
    words = keyword.split()
    translations = {}
    
    for word in words:
        result = get_cui_for_keyword(cursor, word)
        if result:
            cui = result["CUI"]
            translated_rows = get_translations(cursor, cui)
            
            for row in translated_rows:
                lang = row["LAT"]
                translations.setdefault(lang, []).append(row["STR"])
    
    merged_translations = {lang: " ".join(trans) for lang, trans in translations.items()}
    return merged_translations
def get_translation(keyword):
    import mysql.connector
    import json

    # Database connection details
    db_config = {
        'host': '172.16.34.1',
        'port': 3307,
        'user': 'umls',
        'password': 'umls',
        'database': 'umls2024'
    }
    connection = mysql.connector.connect(**db_config)
    cursor = connection.cursor(dictionary=True)
    result = get_cui_for_keyword(cursor, keyword)
        
    if result is None:
        translations = find_and_merge_translations(cursor, keyword)
        output = {"Keyword": keyword, "synonymous": translations}
    else:
        cui = result["CUI"]
        translated_rows = get_translations(cursor, cui)
        translations = {row["LAT"]: row["STR"] for row in translated_rows}
        output = {"Keyword": keyword, "synonymous": translations}
    return output
        
get_translation("erwe")

In [ ]:
import json

# Load the updated keywords file
updated_keywords_path = '/home/mshahidul/project1/all_tran_data/prompt info/updated_keywords.json'
with open(updated_keywords_path, 'r', encoding='utf-8') as file:
    updated_keywords = json.load(file)


# Process each sentence and its keywords
translations_list = []
import tqdm
for item in tqdm.tqdm(updated_keywords):
    sentence = item['sentence']
    keywords = item['keywords']
    dict_keywords={}
    for keyword in keywords:
        translation = get_translation(keyword)
        dict_keywords[translation['Keyword']]=translation['synonymous']
        # translations_list.append(translation)
        # print(translations_list)
    translations_list.append({
        "sentence": sentence,
        "synonymous": dict_keywords
    })



In [ ]:
# Save the results to a new JSON file
output_path = '/home/mshahidul/project1/all_tran_data/prompt info/keyword_synonymous.json'
with open(output_path, 'w', encoding='utf-8') as output_file:
    json.dump(translations_list, output_file, ensure_ascii=False, indent=4)

print(f"Translations saved to {output_path}")

In [ ]:
def get_one_hop_relationships(keyword):
    rel_meanings = {
        "RB": "Broader concept",
        "RN": "Narrower concept",
        "RO": "Other related concept",
        "SY": "Synonym",
        "PAR": "Parent concept",
        "CHD": "Child concept",
        "AQ": "Allowed qualifier",
        "QB": "Qualifier of broader concept",
        "QB": "Qualifier of narrower concept",
        "SIB": "Sibling concept",
        "DEL": "Deleted concept",
        "RQ": "Related and possibly synonymous concept"
    }
    """Retrieve one-hop relationships for a given keyword using UMLS MRREL table."""
    import mysql.connector
    import json

    # Database connection details
    db_config = {
        'host': '172.16.34.1',
        'port': 3307,
        'user': 'umls',
        'password': 'umls',
        'database': 'umls2024'
    }
    connection = mysql.connector.connect(**db_config)
    cursor = connection.cursor(dictionary=True)
    cui = get_cui_for_keyword(cursor, keyword)
    if not cui:
        return None
    else:
        cui=cui['CUI']
    try:
        connection = mysql.connector.connect(**db_config)
        cursor = connection.cursor(dictionary=True)
        
        query = """
        SELECT a.CUI1, a.CUI2, a.REL, a.RELA, b.STR AS CUI2_Name
        FROM MRREL a 
        JOIN MRCONSO b ON a.CUI2 = b.CUI
        WHERE a.CUI1 = %s 
          AND a.STYPE1 = 'CUI'
          AND b.TS = 'P'
          AND b.STT = 'PF'
          AND b.ISPREF = 'Y'
          AND b.LAT = 'ENG'
        """
        cursor.execute(query, (cui,))
        relations = cursor.fetchall()
        unique_relations = []
        seen = set()

        for row in relations:
            relation_tuple = (row["REL"],row["CUI2_Name"])
            if relation_tuple not in seen:
                seen.add(relation_tuple)
                unique_relations.append({"relation": rel_meanings[row["REL"]],"related_concept": row["CUI2_Name"]})
        return { 
            "keyword": keyword,
            "relationships": unique_relations  }
    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return { 
            "keyword": keyword,
            "relationships": []  }
    finally:
        if connection.is_connected():
            connection.close()

get_one_hop_relationships("heart surgery")

In [ ]:
import json

# Define the file path
file_path = '/home/mshahidul/project1/all_tran_data/prompt info/updated_keywords.json'

# Open and read the JSON file
with open(file_path, 'r', encoding='utf-8') as file:
    updated_keywords = json.load(file)
full_res=[]
import tqdm
for x in tqdm.tqdm(updated_keywords):
    res={}
    for x2 in (x['keywords']):
        t=get_one_hop_relationships(x2)
        if t is None:
            res[x2]=[]
        else:
            res[x2]=t['relationships']
    full_res.append({
        "sentence": x['sentence'],
        "relationships": res
    })
# Save the results to a new JSON file
output_path = '/home/mshahidul/project1/all_tran_data/prompt info/keyword_relationships.json'
with open(output_path, 'w', encoding='utf-8') as output_file:
    json.dump(full_res, output_file, ensure_ascii=False, indent=4)


In [ ]:
from openai import OpenAI 
client = OpenAI(api_key=json.load(open('/home/mshahidul/project1/api.json', 'r'))['openai_api'])
import json
def translate(text):
    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                # {"role": "system", "content": f""},
                    {"role": "user", "content": f"Translate each keyword in (French, Portuguese, German, and Spanish). return this in json format(keyword, translated_keyword(French, Portuguese, German, and Spanish)) and remove extra things and keep only json data: {text}"}],
            temperature=0.5
        )
    return (response.choices[0].message.content)

In [ ]:
t=(translate("heart surgery"))

In [ ]:
import json
from openai import OpenAI 

# Load API key
with open('/home/mshahidul/project1/api.json', 'r') as f:
    api_key = json.load(f)['openai_api']

client = OpenAI(api_key=api_key)

def translate(text, max_retries=3):

    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "user", "content": f"Translate the given keyword into French, Portuguese, German, and Spanish. Provide the output strictly in JSON format without any extra text: {{'keyword': '{text}', 'translated_keywords': {{'French': '...', 'Portuguese': '...', 'German': '...', 'Spanish': '...'}}}}"}
            ],
            temperature=0.5
        )
        
    try:    
        t=json.loads((response.choices[0].message.content).replace("```","").replace("json",""))
        return t
    except:
        print("ok")
        from utils import save_to_json
        save_to_json("/home/mshahidul/project1/all_tran_data/prompt_info//errors_keywords.json",response.choices[0].message.content)
          

# Example usage
translated_data = translate("heart surgery")
print(translated_data)



In [7]:
import json
from openai import OpenAI 

# Load API key
with open('/home/mshahidul/project1/api.json', 'r') as f:
    api_key = json.load(f)['openai_api']

client = OpenAI(api_key=api_key)

def KG(text, max_retries=3):

    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "user", "content": f"create knowledge graph for the given keyword. Provide the output strictly in JSON format without any extra text: {text}"}
            ],
            temperature=0.5
        )
        
    try:    
        t=json.loads((response.choices[0].message.content).replace("```","").replace("json",""))
        return t
    except:
        print("ok")
        from utils import save_to_json
        save_to_json("/home/mshahidul/project1/all_tran_data/prompt_info//KG_errors.json",response.choices[0].message.content)
          

# Example usage
# translated_data = translate("heart surgery")
# print(translated_data)



In [8]:
syn_gpt4o=[]
import tqdm
for x in tqdm.tqdm(keywords):
    t=KG(x)
    syn_gpt4o.append({
        "keyword": x,
        "kg": t
    })
import json
output_path = '/home/mshahidul/project1/all_tran_data/prompt_info/kg_gpt4o_mini.json'
with open(output_path, 'w', encoding='utf-8') as output_file:
    json.dump(syn_gpt4o, output_file, ensure_ascii=False, indent=4)

 72%|███████▏  | 312/434 [38:02<14:09,  6.97s/it]

ok


 77%|███████▋  | 336/434 [41:09<14:26,  8.85s/it]

ok


100%|██████████| 434/434 [53:56<00:00,  7.46s/it]


In [ ]:
from utils import compute_bleu_chrf
import json
import tqdm
import os
from utils2 import data_import
from unsloth import FastLanguageModel
from utils2 import keyword_relationships_umls, keyword_synonymous_umls
from utils2 import kg_gpt4o_mini_umls, synonyms_gpt4o_diff_lang_umls,extract_keywords,gpt4o_mini_tran_func

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

original_file,results_data,sentence_to_prompt=data_import()
def find_cod_prompt(english_sentence):
    return sentence_to_prompt.get(english_sentence, "Prompt not found")
###########################
inference_proc="prompt"
def return_prompt_data(text):
    return keyword_synonymous_umls(text)
max_seq_length,dtype,load_in_4bit,model_name = 2048,None ,True, "unsloth/phi-4"
txt="without_finetune_keyword_synonymous_umls"
###########################


print(f'''
###############################               
#  {model_name}
#  {txt}                         
###############################
''')

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = False)

from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)
FastLanguageModel.for_inference(model) 
import re
def extract_translation(text):
    match = re.search(r"Spanish: (.*?)<\|im_end\|>", text)
    if match:
        extracted_text = match.group(1)
        return extracted_text
    else:
        text=text.split("\n")
        text=text[len(text)-1]
        text=text.split("<|im_end|>")[0]
        return text
    
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

FastLanguageModel.for_inference(model) 

def inference(text,context):
    # print (f"{keyword_relationships_umls(text)}\nUsing above context translate the input into English to Spanish: \n\nEnglish: {text}\n\nSpanish: ")
    # exit()
    prompt=f'''
        {context}
        Using above context translate the following text into Spanish:
        {text}
        Spanish: "
        '''
    # prompt=f''' 
    #     Translate the following text into Spanish, given this context:
    #     {keyword_relationships_umls(text)}
    #     Text: {text}
    #     Spanish: "
    #     '''
    messages = [
        {"role": "user", "content": prompt,}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,temperature = 1.0, min_p = 1.0)
    return extract_translation(tokenizer.batch_decode(outputs)[0])

def back_inference(text,context):
    prompt=f'''
        {context}
        Using above context translate the following text into English:
        {text}
        English: "
        '''
    # prompt=f'''
    #     Translate the following text into English, given this context:
    #     {keyword_relationships_umls(text)}
    #     Text: {text}
    #     English: "
    #     '''
    messages = [
        {"role": "user", "content": prompt},]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 100, use_cache = True,temperature = 1.0, min_p = 1.0)
    return extract_translation(tokenizer.batch_decode(outputs)[0])

##################################################


def inference_direct(text):
    prompt=f'''
            Translate the following text into Spanish: 
            {text}
            Spanish:
            '''
    messages = [
        {"role": "user", "content": prompt},]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,temperature = 1.0, min_p = 1.0)
    return extract_translation(tokenizer.batch_decode(outputs)[0])

def back_inference_direct(text):
    prompt=f'''
            Translate the following text into English: 
            {text}
            English:
            '''
    messages = [
        {"role": "user", "content": prompt},]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,temperature = 1.0, min_p = 1.0)
    return extract_translation(tokenizer.batch_decode(outputs)[0])

##################################################

total_score=[]

for line in tqdm.tqdm(original_file):
    try:
        # context=return_prompt_data(line['english'])
        context=keyword_synonymous_umls(line["english"])
        # if inference_proc=="prompt":
        hypothesis_text = inference(line['english'],context)
            # reference_text = line['spanish']
        back_translate = back_inference(hypothesis_text,context)
        # elif inference_proc=="direct":
        #     hypothesis_text = inference_direct(line['english'])
        #     # reference_text = line['spanish']
        #     back_translate = back_inference_direct(hypothesis_text)
        # else:
        #     exit()

        # print(f"yes: {back_translate}")
        score=compute_bleu_chrf(line['english'].lower(), back_translate.lower())  
        total_score.append({
            "original_english": line['english'],
            "original_spanish": line['spanish'],
            "translated_spanish": hypothesis_text,
            "back_translated_english": back_translate,
            "bleu_score": score
        })
    except Exception as e:
        print("Error: ",e)
        continue

tt=model_name.split("/")[1]
avg_bleu_score = sum([x['bleu_score']['bleu_score'] for x in total_score]) / len(total_score)
# txt="without_finetune_and_prompt"

print(f"{tt}_{txt}: {avg_bleu_score:.4f}")

with open(f"/home/mshahidul/project1/all_tran_data/sample/{tt}_{txt}.json", 'w', encoding='utf-8') as json_file:
    json.dump(total_score, json_file, ensure_ascii=False, indent=4)

# with open(f"/home/mshahidul/project1/all_tran_data/sample/{tt}_without_finetune_and_prompt.json", 'w', encoding='utf-8') as json_file:
#     json.dump(total_score, json_file, ensure_ascii=False, indent=4)

In [ ]:
import json

file_path = '/home/mshahidul/project1/all_tran_data/prompt_info/updated_keywords.json'

with open(file_path, 'r', encoding='utf-8') as file:
    updated_keywords = json.load(file)
full_res=[]
import tqdm
for x in tqdm.tqdm(updated_keywords):
    list={}
    for x2 in x['keywords']:
        t=translate(x2)
        try:
            list[x2]=t['translated_keywords']
        except:
            print(t)
    full_res.append({
        "sentence": x['sentence'],
        "keywords":x['keywords'],
        "translated_keywords": list
    })


In [ ]:
# Save the results to a new JSON file
output_path = '/home/mshahidul/project1/all_tran_data/prompt_info/translated_keywords_gpt4o_mini.json'
with open(output_path, 'w', encoding='utf-8') as output_file:
    json.dump(full_res, output_file, ensure_ascii=False, indent=4)

In [ ]:
import json

file_path = '/home/mshahidul/project1/all_tran_data/prompt_info/updated_keywords.json'

with open(file_path, 'r', encoding='utf-8') as file:
    updated_keywords = json.load(file)
keywords = set()
import tqdm
for x in tqdm.tqdm(updated_keywords):
    for x2 in (x['keywords']):
        keywords.add(x2)
keywords=list(keywords)
len(keywords)

    

In [ ]:
import json
client = OpenAI(api_key=json.load(open('/home/mshahidul/project1/api.json', 'r'))['openai_api'])
def synonyms_gpt4o(text):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": f"Provide synonyms for the given keyword in different languages (English, French, Portuguese, German, Spanish). Output strictly in JSON format without any extra text: {text}"}
        ],
        temperature=0.5
    )
    # print(response.choices[0].message.content)
    try:
        t = json.loads(response.choices[0].message.content.replace("```", "").replace("json", ""))
        return t
    except:
        print("Error in parsing JSON response")
        from utils import save_to_json
        save_to_json("/home/mshahidul/project1/all_tran_data/prompt_info/synonyms_gpt4o.json", response.choices[0].message.content)
synonyms_gpt4o("heart surgery")

In [ ]:
syn_gpt4o=[]
import tqdm
for x in tqdm.tqdm(keywords):
    t=synonyms_gpt4o(x)
    syn_gpt4o.append({
        "keyword": x,
        "synonyms": t
    })
import json
output_path = '/home/mshahidul/project1/all_tran_data/prompt_info/synonyms_gpt4o_diff_lang.json'
with open(output_path, 'w', encoding='utf-8') as output_file:
    json.dump(syn_gpt4o, output_file, ensure_ascii=False, indent=4)

In [10]:
import json

file_path_relationships = '/home/mshahidul/project1/all_tran_data/prompt_info/keyword_synonymous_umls.json'

with open(file_path_relationships, 'r', encoding='utf-8') as file:
    keyword_relationships = json.load(file)

print(keyword_relationships[0])

{'sentence': 'If you have a weakened immune system due to AIDS, cancer, transplantation, or corticosteroid use, call your doctor if you develop a cough, fever, or shortness of breath.', 'synonymous': {'weakened immune system': {'POR': 'Anemia Hemolítica Autoimune Anemias hemolíticas auto-imunes Anemias hemolíticas auto-imunes Anemia hemolítica auto-imune Anemia hemolítica auto-imune Anemia hemolítica auto-imune Anemia hemolítica auto-imune Anemia hemolítica auto-imune NE Anemia hemolítica auto-imune NE Anemia hemolítica auto-imune Anemia Hemolítica Autoimunitária Anemia hemolítica autoimune (SOE) Anemia hemolítica autoimune Anemia hemolítica autoimune Anemia hemolítica autoimune Anemia hemolítica autoimune Anemia hemolítica autoimune Anemias hemolíticas autoimunes Anemia hemolítica autoimune SOE Anemias hemolíticas autoimunes Esclerodermia sistémica Esclerose sistémica Esclerose sistémica progressiva Esclerodermia sistémica Esclerose Sistêmica Esclerodermia sistêmica Esclerose sistêmic

In [ ]:
conda create --name unsloth_gemma python=3.11 -y
conda activate unsloth_env
pip3 install torch
pip install "unsloth[cu124-ampere-torch250] @ git+https://github.com/unslothai/unsloth.git"
pip install evaluate sacrebleu
pip install openai
# pip install "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
conda create --name gamma python=3.11 -y
conda activate unsloth_om
pip3 install torch
pip install "unsloth[cu124-ampere-torch250] @ git+https://github.com/unslothai/unsloth.git" evaluate sacrebleu openai

In [ ]:
conda create --name unsloth_env python=3.11 -y
conda activate unsloth_env
pip3 install torch
pip install "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git"
pip install evaluate sacrebleu openai

In [ ]:
mkdir unsloth
cd unsloth
uv venv unsloth --python 3.11
source unsloth/bin/activate
uv pip install mysql-connector-python torch "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git" evaluate sacrebleu openai
deactivate

In [ ]:
mkdir unsloth
cd unsloth
uv venv unsloth --python 3.11
source unsloth/bin/activate
uv pip install mysql-connector-python torch "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git" evaluate sacrebleu openai
deactivate

In [ ]:
## for omega
conda create --name gamma python=3.11 -y && conda activate gamma
uv pip install mysql-connector-python evaluate sacrebleu openai
uv pip install torch
uv pip install "unsloth[cu124-torch260] @ git+https://github.com/unslothai/unsloth.git" 
unset LD_LIBRARY_PATH

In [ ]:
uv pip install torch --index-url https://download.pytorch.org/whl/cu121